In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

CRdata = pd.read_csv('CRdataShared.csv')
pd.set_option('display.max_columns', None)

In [48]:
#Exploring the counts of the survey completions
average_fills = CRdata['ID'].value_counts().mean()
print(f"The average number of times a user filled in the survey is {average_fills:.2f}")
less_than_10 = CRdata['ID'].value_counts()[CRdata['ID'].value_counts() < 10].count()
more_than_10 = CRdata['ID'].value_counts()[CRdata['ID'].value_counts() >= 10].count()

print(f"Number of IDs that filled in the survey less than 10 times: {less_than_10}")
print(f"Number of IDs that filled in the survey more than 10 times: {more_than_10}")

The average number of times a user filled in the survey is 30.10
Number of IDs that filled in the survey less than 10 times: 165866
Number of IDs that filled in the survey more than 10 times: 118160


In [49]:
#Date formatting
day = CRdata['date'].str[:2]
month = CRdata['date'].str[2:5]
year = CRdata['date'].str[5:]

month_mapping = {'apr': '04', 'may': '05', 'jun': '06', 'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12', 'jan': '01', 'feb': '02', 'mar': '03'}
month = month.map(month_mapping)
actual_date = pd.to_datetime(day.astype(str) + '/' + month + '/' + year.astype(str), format='%d/%m/%Y')
CRdata['actual_date'] = actual_date
CRdata['weekday'] = CRdata['actual_date'].dt.day_name()

#Filter out weekend
CRdata = CRdata[(CRdata['weekday'] != 'Sunday') & (CRdata['weekday'] != 'Monday')]

#Filter out age categories that do not go to work
agelist = ['19-29', '30-39', '40-49', '50-59', '60-69'] #List with age categories that go to work
CRdata = CRdata[CRdata['age'].isin(agelist)]


In [50]:
#Filter on number of survey completions
treshold = 10
ID_count = CRdata['ID'].value_counts()
CRdata_treshold = CRdata[CRdata['ID'].isin(ID_count[ID_count > treshold].index)]
CRdata_treshold

,gender,age,profession,contact,cough,sorethroat,fever,shortofbreath,bowelsymptoms,chestsymptoms,eyesymptoms,lossofsmell,fatigue,headache,vaccine,outhousework,outofhouse,numberpersons5m,numbervisitors,numberpersons150cm,numberpersons150cmspoken,mask,public,holiday,brainfog,depressed,date,fatigueExtreme,sport,ID,positivetest,numbertimescovid,firstpositivetest,dayssurroundingpositivetest,everpositive,negativetest,numbernegativetest,firstnegativetest,dayssurroundingnegativetest,evernegativetest,evercovidinapp,evercovidoutapp,actual_date,weekday
26,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,1.0,0,0,2,0,NaN,NaN,NaN,NaN,NaN,07apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-07,Tuesday
27,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,3.0,2,0,0,0,NaN,NaN,NaN,NaN,NaN,08apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-08,Wednesday
30,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,1.0,0,0,3,0,NaN,NaN,NaN,NaN,NaN,15apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-15,Wednesday
31,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,3.0,1,0,4,0,NaN,NaN,NaN,NaN,NaN,17apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-17,Friday
32,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,3.0,0,0,6,0,NaN,NaN,NaN,NaN,NaN,18apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-18,Saturday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8550443,f,50-59,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,9.0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,14may2020,NaN,NaN,284021,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-05-14,Thursday
8550444,f,50-59,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,4.0,1,0,2,0,NaN,NaN,NaN,NaN,NaN,19may2020,NaN,NaN,284021,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-05-19,Tuesday
8550445,f,50-59,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,0.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,23may2020,NaN,NaN,284021,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-05-23,Saturday
8550447,f,50-59,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,1,7.0,2,1,10,0,NaN,NaN,NaN,NaN,NaN,06jun2020,NaN,NaN,284021,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-06-06,Saturday


In [52]:
#Remove participants with non-office professions
profession_list = ['other_contact', 'hcpro', 'student', 'pensioner']
CR_profession = CRdata_treshold[~CRdata_treshold['profession'].isin(profession_list)]
CR_profession['#Survey completions'] = 1 #Add column to later count the number of survey completions
CR_profession.head()

C:\Users\20202719\AppData\Local\Temp\ipykernel_37200\4141412807.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CR_profession['#Survey completions'] = 1 #Add column to later count the number of survey completions


,gender,age,profession,contact,cough,sorethroat,fever,shortofbreath,bowelsymptoms,chestsymptoms,eyesymptoms,lossofsmell,fatigue,headache,vaccine,outhousework,outofhouse,numberpersons5m,numbervisitors,numberpersons150cm,numberpersons150cmspoken,mask,public,holiday,brainfog,depressed,date,fatigueExtreme,sport,ID,positivetest,numbertimescovid,firstpositivetest,dayssurroundingpositivetest,everpositive,negativetest,numbernegativetest,firstnegativetest,dayssurroundingnegativetest,evernegativetest,evercovidinapp,evercovidoutapp,actual_date,weekday,#Survey completions
26,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,1.0,0,0,2,0,NaN,NaN,NaN,NaN,NaN,07apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-07,Tuesday,1
27,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,3.0,2,0,0,0,NaN,NaN,NaN,NaN,NaN,08apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-08,Wednesday,1
30,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,1.0,0,0,3,0,NaN,NaN,NaN,NaN,NaN,15apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-15,Wednesday,1
31,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,3.0,1,0,4,0,NaN,NaN,NaN,NaN,NaN,17apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-17,Friday,1
32,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,3.0,0,0,6,0,NaN,NaN,NaN,NaN,NaN,18apr2020,NaN,NaN,2,0,0,0,NaN,0,0,0,0,NaN,0,NaN,0,2020-04-18,Saturday,1


### Formatting the data

In [53]:
CR_profession['outhousework'] = CR_profession['outhousework'].astype(int)

#Sort dataframe by ID and dates
CR_profession_sorted = CR_profession.sort_values(by=['ID', 'actual_date'])

CR_profession_sorted['test'] = CR_profession_sorted.apply(lambda row: 1 if row['positivetest'] == 1 else (0 if row['negativetest'] == 1 else np.nan), axis=1)

C:\Users\20202719\AppData\Local\Temp\ipykernel_37200\3353941076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CR_profession['outhousework'] = CR_profession['outhousework'].astype(int)


In [56]:

#Funtion for calculating the infection range
def infection_range(row):
    if pd.isnull(row['test']):
        return np.nan
    else:
        test_date = row['actual_date'] #Retrieve the date of the test indication
        start_date = test_date - pd.Timedelta(weeks=3)
        infection_date = test_date - pd.Timedelta(weeks=1) #Define te range of infection
        date_range = pd.date_range(start=start_date, end=infection_date) 

        return date_range



CR_profession_sorted['infection_range'] = CR_profession_sorted.apply(infection_range, axis=1) #Apply the function to the dataframe

In [55]:
#Filter IDs that have tested at least once
tested_ids = CR_profession_sorted[(CR_profession_sorted['positivetest'] == 1) | (CR_profession_sorted['negativetest'] == 1)]['ID'].unique()
CRdata_tested = CR_profession_sorted[CR_profession_sorted['ID'].isin(tested_ids)]
CRdata_tested.head()

,gender,age,profession,contact,cough,sorethroat,fever,shortofbreath,bowelsymptoms,chestsymptoms,eyesymptoms,lossofsmell,fatigue,headache,vaccine,outhousework,outofhouse,numberpersons5m,numbervisitors,numberpersons150cm,numberpersons150cmspoken,mask,public,holiday,brainfog,depressed,date,fatigueExtreme,sport,ID,positivetest,numbertimescovid,firstpositivetest,dayssurroundingpositivetest,everpositive,negativetest,numbernegativetest,firstnegativetest,dayssurroundingnegativetest,evernegativetest,evercovidinapp,evercovidoutapp,actual_date,weekday,#Survey completions,test
1265,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,2.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,07apr2020,NaN,NaN,33,0,0,0,-94.0,1,0,0,0,NaN,0,0.0,0,2020-04-07,Tuesday,1,NaN
1266,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,2.0,1,0,1,0,NaN,NaN,NaN,NaN,NaN,08apr2020,NaN,NaN,33,0,0,0,-93.0,1,0,0,0,NaN,0,0.0,0,2020-04-08,Wednesday,1,NaN
1267,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,2.0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,10apr2020,NaN,NaN,33,0,0,0,-91.0,1,0,0,0,NaN,0,0.0,0,2020-04-10,Friday,1,NaN
1270,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,2.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,14apr2020,NaN,NaN,33,0,0,0,-87.0,1,0,0,0,NaN,0,0.0,0,2020-04-14,Tuesday,1,NaN
1271,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,2.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,15apr2020,NaN,NaN,33,0,0,0,-86.0,1,0,0,0,NaN,0,0.0,0,2020-04-15,Wednesday,1,NaN


In [24]:
from tqdm import tqdm

#Filter out the rows that are not in the infection range, per participant
id_list = CRdata_tested['ID'].unique()
df_final = pd.DataFrame()
for id in tqdm(id_list):
    infection_range = CRdata_tested[CRdata_tested['ID'] == id]['infection_range']
    try:
        infection_range_notnull = infection_range[infection_range.notnull()].values[0]
    except:
        infection_range_notnull = None
    if infection_range_notnull is None:
        continue
    else:
        df_id = CRdata_tested[CRdata_tested['ID'] == id]
        df_id['test'] = df_id['test'].max()
        df_within_infection_range = df_id[df_id['actual_date'].isin(infection_range_notnull)]

        df_final = pd.concat([df_final, df_within_infection_range])


df_final


  0%|          | 0/10620 [00:00<?, ?it/s]C:\Users\20202719\AppData\Local\Temp\ipykernel_37200\23004130.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id['test'] = df_id['test'].max()
100%|██████████| 10620/10620 [05:29<00:00, 32.25it/s]


,gender,age,profession,contact,cough,sorethroat,fever,shortofbreath,bowelsymptoms,chestsymptoms,eyesymptoms,lossofsmell,fatigue,headache,vaccine,outhousework,outofhouse,numberpersons5m,numbervisitors,numberpersons150cm,numberpersons150cmspoken,mask,public,holiday,brainfog,depressed,date,fatigueExtreme,sport,ID,positivetest,numbertimescovid,firstpositivetest,dayssurroundingpositivetest,everpositive,negativetest,numbernegativetest,firstnegativetest,dayssurroundingnegativetest,evernegativetest,evercovidinapp,evercovidoutapp,actual_date,weekday,#Survey completions,year_weeknumber,monthnumber_year,WFH_rule_applied,outhouse_WFHapplied,outhouse_WFHnot,hoursouthouse_WFHapplied,hoursouthouse_WFHnot,test,infection_range
1331,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,0.0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,19jun2020,NaN,NaN,33,0,0,0,-21.0,1,0,0,0,NaN,0,0.0,0,2020-06-19,Friday,1,202024,062020,0,0,0,0.0,0.0,1.0,NaN
1332,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,3.0,4,2,0,0,NaN,NaN,NaN,NaN,NaN,20jun2020,NaN,NaN,33,0,0,0,-20.0,1,0,0,0,NaN,0,0.0,0,2020-06-20,Saturday,1,202024,062020,0,0,0,0.0,3.0,1.0,NaN
1334,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,2.0,4,0,0,0,NaN,NaN,NaN,NaN,NaN,23jun2020,NaN,NaN,33,0,0,0,-17.0,1,0,0,0,NaN,0,0.0,0,2020-06-23,Tuesday,1,202025,062020,0,0,0,0.0,2.0,1.0,NaN
1335,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,2.0,4,2,0,0,NaN,NaN,NaN,NaN,NaN,25jun2020,NaN,NaN,33,0,0,0,-15.0,1,0,0,0,NaN,0,0.0,0,2020-06-25,Thursday,1,202025,062020,0,0,0,0.0,2.0,1.0,NaN
1338,m,60-69,other,0,0,0,0,0,0,0,0.0,0,NaN,NaN,NaN,0,0.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,30jun2020,NaN,NaN,33,0,0,0,-10.0,1,0,0,0,NaN,0,0.0,0,2020-06-30,Tuesday,1,202026,062020,0,0,0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8549059,m,60-69,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,NaN,0,0.0,0,0,0,0,0.0,0.0,0,NaN,NaN,16feb2021,0.0,0.0,283985,0,0,0,-371.0,1,0,0,0,-7.0,1,1.0,0,2021-02-16,Tuesday,1,202107,022021,1,0,0,0.0,0.0,1.0,NaN
8549862,f,50-59,education,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0,2.0,2,0,6,6,2.0,1.0,0,NaN,NaN,03apr2021,0.0,0.0,284003,0,0,0,NaN,0,0,0,0,-19.0,1,0.0,0,2021-04-03,Saturday,1,202113,042021,1,0,0,2.0,0.0,0.0,NaN
8549863,f,50-59,education,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0.0,0,1.0,2,1,1,1,2.0,1.0,0,NaN,NaN,06apr2021,0.0,0.0,284003,0,0,0,NaN,0,0,0,0,-16.0,1,0.0,0,2021-04-06,Tuesday,1,202114,042021,1,0,0,1.0,0.0,0.0,NaN
8549864,f,50-59,education,0,0,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1,8.0,2,0,12,12,2.0,1.0,0,NaN,NaN,10apr2021,0.0,0.0,284003,0,0,0,NaN,0,0,0,0,-12.0,1,0.0,0,2021-04-10,Saturday,1,202114,042021,1,1,0,8.0,0.0,0.0,NaN


In [25]:
#Check for duplicate IDs
duplicate_ids = df_final[df_final['infection_range'].notnull()]['ID'].value_counts()
duplicate_ids = duplicate_ids[duplicate_ids > 1].index
duplicate_ids

Index([], dtype='int64', name='ID')

In [31]:
df_withoutNAN = df_final[df_final['profession'].notna()] #Create dataset without professios NAN values
df_withoutNAN['ID'].nunique()

4446

In [35]:
agg_dict = {'outofhouse': 'mean',  #hours out of house yesterday (Not implemented)
            'numberpersons150cm' : 'mean', #number of persons within 1.5m yesterday
            'numberpersons150cmspoken' : 'mean', #number of persons within 1.5m spoken to yesterday
            'numberpersons5m' : 'mean', #number of persons within 5m yesterday
            'outhousework': 'mean', #mean number of times working out of the house 
            'test' : 'max', #Y value
            '#Survey completions' : 'sum' #number of survey completions
            }

trainingdata = df_final.groupby('ID').agg(agg_dict) #Group the data by ID to get risk profiles
trainingdata

,outofhouse,numberpersons150cm,numberpersons150cmspoken,numberpersons5m,outhousework,test,#Survey completions
ID,,,,,,,
33,1.428571,0.428571,0.428571,2.714286,0.000000,1.0,7
88,3.000000,1.666667,1.666667,1.000000,0.333333,1.0,3
94,1.857143,3.428571,0.142857,2.285714,0.000000,0.0,7
98,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,1
120,4.000000,1.000000,1.000000,1.000000,0.500000,0.0,2
...,...,...,...,...,...,...,...
283835,3.500000,3.100000,0.900000,1.800000,0.000000,0.0,10
283890,9.000000,10.000000,10.000000,2.000000,1.000000,0.0,1
283907,2.000000,1.000000,0.000000,1.000000,1.000000,1.0,1


In [37]:
second_treshold = 5 
trainingdata = trainingdata[trainingdata['#Survey completions'] > second_treshold] #FIlter the aggregation on number of survey completions = 5
trainingdata.to_csv('trainingdata.csv') 

In [38]:
trainingdata_nan = df_withoutNAN.groupby('ID').agg(agg_dict)
trainingdata_nan = trainingdata_nan[trainingdata_nan['#Survey completions'] > second_treshold]
trainingdata_nan.to_csv('trainingdata_nan.csv') #Create the data set without the NAN professions